In [3]:
import os
import sys
import pickle

### Create dataset for MBPP

In [169]:
problems = pickle.load(open('/hcode/mbpp_problems.pickle', 'rb'))

In [170]:
def remove_few_show(p):
    c = problems['MBPP/0']['prompt'].find('[DONE]')
    return p[c:].replace('[BEGIN]', '').replace('[DONE]', '').strip('\n')

In [171]:
general_prompt = problems['MBPP/0']['code'] + '\n[BEGIN]\n' + remove_few_show(problems['MBPP/0']['prompt']) + '\n[DONE]\n'
general_prompt += problems['MBPP/1']['code'] + '\n[BEGIN]\n' + remove_few_show(problems['MBPP/1']['prompt']) + '\n[DONE]\n'
general_prompt += problems['MBPP/2']['code'] + '\n[BEGIN]\n' + remove_few_show(problems['MBPP/2']['prompt']) + '\n[DONE]\n'

In [172]:
print(general_prompt)

R = 3
C = 3
def min_cost(cost, m, n): 
	tc = [[0 for x in range(C)] for x in range(R)] 
	tc[0][0] = cost[0][0] 
	for i in range(1, m+1): 
		tc[i][0] = tc[i-1][0] + cost[i][0] 
	for j in range(1, n+1): 
		tc[0][j] = tc[0][j-1] + cost[0][j] 
	for i in range(1, m+1): 
		for j in range(1, n+1): 
			tc[i][j] = min(tc[i-1][j-1], tc[i-1][j], tc[i][j-1]) + cost[i][j] 
	return tc[m][n]
[BEGIN]
You are an expert Python programmer, and here is your task: Write a function to find the minimum cost path to reach (m, n) from (0, 0) for the given cost matrix cost[][] and a position (m, n) in cost[][]. Your code should pass these tests:

assert min_cost([[1, 2, 3], [4, 8, 2], [1, 5, 3]], 2, 2) == 8
assert min_cost([[2, 3, 4], [5, 9, 3], [2, 6, 4]], 2, 2) == 12
assert min_cost([[3, 4, 5], [6, 10, 4], [3, 7, 5]], 2, 2) == 16
[DONE]
def similar_elements(test_tup1, test_tup2):
  res = tuple(set(test_tup1) & set(test_tup2))
  return (res) 
[BEGIN]
You are an expert Python programmer, and here is your task: 

In [173]:
len(general_prompt)

1761

In [174]:
NUM_TASKS = 510

In [175]:
tasks = []

for i in range(NUM_TASKS):
    results_code1 = pickle.load(open('/mbpp_results2/%d_code.pickle' % i, 'rb'))

    subtasks = []
    
    for seed in range(1):
        for k in range(5):
            generated_solution = results_code1[('MBPP/%d' % i, seed)][0][k]

            print('------------------')
            print(generated_solution)
            interrogate_prompt = general_prompt + generated_solution + '\n[BEGIN]\n'
            correct_prompt = remove_few_show(problems['MBPP/%d' % i]['prompt'])
            subtasks.append((interrogate_prompt, correct_prompt))

    tasks.append((i, subtasks))

------------------

def min_cost(cost, m, n):
  dp = [[0 for i in range(n+1)] for i in range(m+1)]
  for i in range(m+1):
    for j in range(n+1):
      if i == 0 or j == 0:
        dp[i][j] = 0
      elif i==1 and j==1:
        dp[i][j] = cost[0][0]
      else:
        dp[i][j] = min(dp[i-1][j], dp[i][j-1]) + cost[i-1][j-1]
  return dp[m][n]

------------------

def min_cost(cost, m, n):
  if m == 0 and n == 0:
    return cost[0][0]
  elif m == 0 or n == 0:
    return float('inf')
  return min(min_cost(cost, m - 1, n), min_cost(cost, m, n - 1)) + cost[m][n]

------------------

from heapq import heapify, heappop

def min_cost(cost_mat, m, n):
  # Your code here
  dist = [[0 for x in range(m)] for y in range(n)]
  for i in range(n):
    dist[i][0] = cost_mat[i][0]
  for i in range(m):
    dist[0][i] = cost_mat[0][i]
  for i in range(1, n):
    for j in range(1, m):
      dist[i][j] = min(dist[i-1][j], dist[i][j-1], dist[i-1][j-1]) + cost_mat[i][j]
  return dist[n-1][m-1]

-------------

In [176]:
pickle.dump(tasks, open('interrogate_tasks_mbpp.pickle', 'wb'))

### Create dataset for HE

In [177]:
problems = pickle.load(open('/he_problems.pickle', 'rb'))

In [178]:
general_prompt = problems['HumanEval/0']['canonical_solution'] + '[BEGIN]\n' + problems['HumanEval/0']['prompt'] + '[DONE]\n'
general_prompt += problems['HumanEval/1']['canonical_solution'] + '[BEGIN]\n' + problems['HumanEval/1']['prompt'] + '[DONE]\n'
general_prompt += problems['HumanEval/2']['canonical_solution'] + '[BEGIN]\n' + problems['HumanEval/2']['prompt'] + '[DONE]\n'

In [179]:
len(general_prompt)

1925

In [180]:
NUM_TASKS = 164

In [181]:
tasks = []

for i in range(NUM_TASKS):
    results_code1 = pickle.load(open('/he_results2/%d_code.pickle' % i, 'rb'))

    subtasks = []
    
    for seed in range(5):
        for k in range(5):
            generated_solution = results_code1[('HumanEval/%d' % i, seed)][0][k]

            interrogate_prompt = general_prompt + generated_solution + '[BEGIN]\n'
            correct_prompt = problems['HumanEval/%d' % i]['prompt']
            subtasks.append((interrogate_prompt, correct_prompt))

    tasks.append((i, subtasks))

In [182]:
pickle.dump(tasks, open('interrogate_tasks.pickle', 'wb'))